CHANGELOG

v0.3
- hold out and external testing

v0.2 11/1/2023
- load each shard from data raw one by one instead of loading it all together into one dataset and then using dataset.skip.take

v0.1 10/26/2023
- try not sharding because it keeps crashing and changing the size of the shard doesn't matter

v0.0 10/27/2023
- A lot of patches are all black (from padding with 0's) or have mostly black and some white (background of slide)
- Use filter_data to crudely get rid of as many as possible

In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, pandas as pd, tensorflow as tf, matplotlib.pyplot as plt, skimage.io, os, gc
from tensorflow.keras import Input, Model, models, layers, optimizers, losses, callbacks, utils
from pathlib import Path
from jarvis.utils.display import imshow
from PIL import Image
from skimage.transform import rescale, resize
from histomicstk.preprocessing.augmentation import rgb_perturb_stain_concentration
from IPython.display import HTML, Javascript, display

Image.MAX_IMAGE_PIXELS = None

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset

[ 2023-11-19 04:20:09 ] CUDA_VISIBLE_DEVICES automatically set to: 3           


In [2]:
root = '/home/jjlou/Jerry/wsi-arterio/vessel_detection_and_rough_segmentation/data_test'

In [3]:
def filter_data(dataset=None):
    dats = []
    masks = []
    for d, m in dataset:
        rgb_mean = tf.math.reduce_mean(d)
        if (rgb_mean > 210) or (rgb_mean < 65) or (rgb_mean == None):
            continue
        dats.append(d)
        masks.append(m)
    
    return tf.data.Dataset.from_tensor_slices((dats, masks))

In [4]:
def process_data(load=None, save_path=None):
    shards_path = sorted(glob.glob(f'{load}/*/*'))
    for s in shards_path:
        ID = s.split('/')[-2]
        shard_num = s.split('/')[-1]
        save_file = f'{save_path}/{ID}/{shard_num}'
        if not glob.glob(f'{save_file}/*/*/*.snapshot'):
            assert not glob.glob(f'{save_file}/*/*.shard'), f'{save_file} did not save'
            shard = tf.data.Dataset.load(s)
            shard = filter_data(shard)
            shard.save(save_file)
            restart_kernel()
            time.sleep(10)
        else:
            continue

In [5]:
train_load = f'{root}/train'
train_save = f'{root}/train_filtered'

if len(glob.glob(f'{train_save}/*/*')) < len(glob.glob(f'{train_load}/*/*')):
    process_data(load=train_load, save_path=train_save)

In [6]:
hold_load = f'{root}/hold_out'
hold_save = f'{root}/hold_out_filtered'

if len(glob.glob(f'{hold_save}/*/*')) < len(glob.glob(f'{hold_load}/*/*')):
    process_data(load=hold_load, save_path=hold_save)

In [7]:
external_load = f'{root}/external'
external_save = f'{root}/external_filtered'

if len(glob.glob(f'{external_save}/*/*')) < len(glob.glob(f'{external_load}/*/*')):
    process_data(load=external_load, save_path=external_save)